In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master("local[*]").appName("Datamanipulation").getOrCreate()

In [6]:
spark

TO ANALYZE DATA, THERE ARE BUNCH OF QUESTIONS, I WILL ANSWER IT BY USING PYSPARK

In [33]:
# read our data - lives in a csv file

df = spark.read.option("header","true").csv("/content/drive/MyDrive/Sample - EU Superstore.csv")

In [11]:
# how many rows of the EU Superstore dataset have the country being France
df.filter(df.Country=="France").count()

2827

In [12]:
# of those, how many are profitable?
df.filter((df.Country=="France")&(df.Profit>0)).count()


2277

In [13]:
# how many different discount brackets exist? what are they?
discount_brackets= df.select("discount").distinct().orderBy("discount")
print("There are " +str(discount_brackets.count()) +  " different discount brackets. These are as below:")
discount_brackets.show()



There are 14 different discount brackets. These are as below:
+--------+
|discount|
+--------+
|       0|
|     0.1|
|    0.15|
|     0.2|
|     0.3|
|    0.35|
|     0.4|
|    0.45|
|     0.5|
|     0.6|
|    0.65|
|     0.7|
|     0.8|
|    0.85|
+--------+



In [14]:
# let's see the total profit by discount bracket, make sure they are ordered by total profit
from pyspark.sql.functions import *

profit_by_discount = df.groupBy("discount").agg(round(sum("profit")).alias("profit")).orderBy("discount")
print("Total profit by discount bracket:")
profit_by_discount.show()

Total profit by discount bracket:
+--------+--------+
|discount|  profit|
+--------+--------+
|       0|383807.0|
|     0.1|126884.0|
|    0.15| 24678.0|
|     0.2|  2190.0|
|     0.3|  -758.0|
|    0.35| -9123.0|
|     0.4|-21346.0|
|    0.45| -1103.0|
|     0.5|-96632.0|
|     0.6|-20517.0|
|    0.65| -6222.0|
|     0.7| -5497.0|
|     0.8|  -460.0|
|    0.85| -3069.0|
+--------+--------+



In [15]:
# what is the value after which we should stop offering discount?
stop_discount = profit_by_discount.filter(profit_by_discount.profit>0).select(max("discount")).show()


+-------------+
|max(discount)|
+-------------+
|          0.2|
+-------------+



In [29]:
# who are the top 5 most profitable customers
df.select("Customer Name","Customer ID","Profit").groupBy("Customer Name","Customer ID").agg({"Profit":"sum"}).orderBy("sum(Profit)",ascending=False).show(5)

+-----------------+-----------+------------------+
|    Customer Name|Customer ID|       sum(Profit)|
+-----------------+-----------+------------------+
|     Susan Pistek|   SP-20920| 4974.512999999999|
|    Patrick Jones|   PJ-18835|3986.0039999999995|
|Patrick O'Donnell|   PO-18865|          3778.197|
|    Ellis Ballard|   EB-13840|           3459.66|
|  Mike Gockenbach|   MG-18145|3144.4439999999995|
+-----------------+-----------+------------------+
only showing top 5 rows



In [32]:
# get all the rows belonging to those 5 customer names: hint, you may need the collect method - how many rows are they?
top_customers = ["Susan Pistek", "Patrick Jones", "Patrick O'Donnell", "Ellis Ballard", "Mike Gockenbach"]
top_customers_df = df.filter(df["Customer Name"].isin(top_customers))
rows = top_customers_df.collect()
print("Number of rows for top customers: ", len(rows))



Number of rows for top customers:  76


In [35]:
# create a new column which is the value of the sale were there not discount applied. Hint: orginal = sales/(1-d)
df = df.withColumn("Original Sales", col("Sales") / (1 - col("Discount")))
df.select("Original Sales","Sales").show()


+------------------+--------+
|    Original Sales|   Sales|
+------------------+--------+
|              79.2|    79.2|
|            388.92|  388.92|
|             35.19|   35.19|
|             50.94|   50.94|
|            307.44|  307.44|
|             122.4|   122.4|
|            413.82|  413.82|
|            428.22|  428.22|
|           3979.29| 3979.29|
|             43.56|   43.56|
|             25.26|   25.26|
|2715.4500000000003|2443.905|
|             12.21|   12.21|
|2549.7599999999998|2167.296|
|            153.45| 138.105|
|142.64999999999998| 128.385|
|            690.12|  690.12|
|              8.16|    8.16|
|            347.88|  347.88|
| 639.4499999999999| 575.505|
+------------------+--------+
only showing top 20 rows



In [37]:
# calculate the difference between sales and discount value

df = df.withColumn("Discount Difference", col("Original Sales") - col("Sales"))

df.select("Original Sales","Sales","Discount Difference").show()



+------------------+--------+-------------------+
|    Original Sales|   Sales|Discount Difference|
+------------------+--------+-------------------+
|              79.2|    79.2|                0.0|
|            388.92|  388.92|                0.0|
|             35.19|   35.19|                0.0|
|             50.94|   50.94|                0.0|
|            307.44|  307.44|                0.0|
|             122.4|   122.4|                0.0|
|            413.82|  413.82|                0.0|
|            428.22|  428.22|                0.0|
|           3979.29| 3979.29|                0.0|
|             43.56|   43.56|                0.0|
|             25.26|   25.26|                0.0|
|2715.4500000000003|2443.905|  271.5450000000001|
|             12.21|   12.21|                0.0|
|2549.7599999999998|2167.296| 382.46399999999994|
|            153.45| 138.105| 15.344999999999999|
|142.64999999999998| 128.385| 14.264999999999986|
|            690.12|  690.12|                0.0|


In [51]:
# how much money did we not gain due to the discounts - per discount bracket?
from pyspark.sql.functions import round

discount_loss = df.groupBy(round("Discount", 2).alias("Discount Bracket")).agg({"Discount Amount": "sum"}).orderBy(sum("Discount Amount"),ascending=False)

discount_loss.show()


+----------------+--------------------+
|Discount Bracket|sum(Discount Amount)|
+----------------+--------------------+
|             0.5|  183734.26500000045|
|             0.1|   84712.44899999996|
|             0.4|   46724.68800000001|
|            0.15|   45233.17650000002|
|             0.6|   39644.04599999997|
|            0.35|  29163.099000000002|
|            0.65|          12219.6555|
|             0.2|  10653.119999999999|
|             0.7|   8534.084999999997|
|            0.85|            4515.438|
|             0.3|            2630.241|
|            0.45|           2083.4415|
|             0.8|   635.6640000000002|
|             0.0|                 0.0|
+----------------+--------------------+



In [52]:
# find the discount bracket which made us not gain the most (dynamically)
max_discount_loss = discount_loss.first()["Discount Bracket"]

print("The discount bracket that caused us to lose the most is: ", max_discount_loss)



The discount bracket that caused us to lose the most is:  0.5


In [53]:
# what would have been the total profit if we removed all orders from that discount group? 

df_no_max_loss = df.filter(df.Discount != max_discount_loss)

total_profit = df_no_max_loss.agg(sum("Profit")).collect()[0][0]

print("The total profit without the discount bracket that caused us to lose the most is: ", total_profit)


The total profit without the discount bracket that caused us to lose the most is:  469461.8565000003


In [56]:
#how much more (or less) profit is that?
total_profit_all = df.agg(sum("Profit")).collect()[0][0]
profit_difference = total_profit_all - total_profit
print("The difference in profit is: ", profit_difference, "So it is less profit with this discount bracket"if profit_difference<0 else "So it is more profit with this discount bracket")





The difference in profit is:  -96632.11499999976 So it is less profit with this discount bracket


In [59]:
# create a temporary table for our superstore table in sql
df.createOrReplaceTempView("superstore_table")


In [60]:
# use an SQL query to count the number of rows
count = spark.sql("SELECT COUNT(*) FROM superstore_table").collect()[0][0]
print("The number of rows in the superstore table is:", count)


The number of rows in the superstore table is: 10000


In [63]:
# Use an SQL query to calculate the profit ratio for each country: hint, ratio is sum(profit)/sum(sales)

query = """
    SELECT Country, ROUND(SUM(Profit)/SUM(Sales), 2) AS profit_ratio
    FROM superstore_table
    GROUP BY Country
    ORDER BY profit_ratio DESC
"""

result = spark.sql(query)

result.show()


+--------------+------------+
|       Country|profit_ratio|
+--------------+------------+
|   Switzerland|        0.29|
|       Austria|        0.26|
|        Norway|        0.25|
|       Belgium|        0.24|
|United Kingdom|        0.21|
|       Finland|        0.19|
|         Spain|        0.19|
|       Germany|        0.17|
|        France|        0.13|
|         Italy|        0.07|
|       Ireland|       -0.44|
|       Denmark|        -0.5|
|   Netherlands|       -0.53|
|        Sweden|       -0.57|
|      Portugal|       -0.58|
+--------------+------------+



In [65]:
# is the country with the largest profit ratio, the country with the largest profit?

query_profit = """
    SELECT Country, SUM(Profit) AS total_profit
    FROM superstore_table
    GROUP BY Country
    ORDER BY total_profit DESC
"""

result_profit = spark.sql(query_profit)

print("Country with the largest profit ratio:")
result.show(1)
print("Country with the largest profit:")
result_profit.show(1)
print("So the country with the largest profit ratio  is not necessarily the country with the largest profit.")



Country with the largest profit ratio:
+-----------+------------+
|    Country|profit_ratio|
+-----------+------------+
|Switzerland|        0.29|
+-----------+------------+
only showing top 1 row

Country with the largest profit:
+--------------+------------------+
|       Country|      total_profit|
+--------------+------------------+
|United Kingdom|111900.15000000001|
+--------------+------------------+
only showing top 1 row

So the country with the largest profit ratio  is not necessarily the country with the largest profit.
